In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !pip install torch-geometric \
#   torch-sparse==latest+cu101 \
#   torch-scatter==latest+cu101 \
#   torch-cluster==latest+cu101 \
#   -f https://pytorch-geometric.com/whl/torch-1.5.0.html

In [0]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
np.random.seed(42)

In [0]:
import os
os.chdir('/content/drive/My Drive/RecSys')

In [0]:
os.listdir('/content/drive/My Drive/RecSys')

['yoochoose-buys.dat',
 'yoochoose-test.dat',
 'yoochoose-clicks.dat',
 'yoochoose_click_binary_100000_sess.dataset',
 'raw']

In [0]:
# df = pd.read_csv('/content/drive/My Drive/RecSys/yoochoose_click_binary_100000_sess.dataset', header=None)
# df
# df.columns=['session_id','timestamp','item_id','category']

In [0]:
df = pd.read_csv('/content/drive/My Drive/RecSys/yoochoose-clicks.dat', header=None)
# df
df.columns=['session_id','timestamp','item_id','category']

In [0]:
df.head()

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,214536502.0,0.0
1,1,2014-04-07T10:54:09.868Z,214536500.0,0.0
2,1,2014-04-07T10:54:46.998Z,214536506.0,0.0
3,1,2014-04-07T10:57:00.306Z,214577561.0,0.0
4,2,2014-04-07T13:56:37.614Z,214662742.0,0.0


In [0]:
df['session_id'].min(),df['session_id'].max()

(1, 4726783)

In [0]:
df.shape

(14012414, 4)

In [0]:
buy_df = pd.read_csv('/content/drive/My Drive/RecSys/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head()

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06T18:44:58.314Z,214537888,12462,1
1,420374,2014-04-06T18:44:58.325Z,214537850,10471,1
2,281626,2014-04-06T09:40:13.032Z,214535653,1883,1
3,420368,2014-04-04T06:13:28.848Z,214530572,6073,1
4,420368,2014-04-04T06:13:28.858Z,214835025,2617,1


In [0]:
buy_df.shape

(1150753, 5)

In [0]:
buy_df['session_id'].min(),buy_df['session_id'].max()

(11, 11562121)

In [0]:
df [df['session_id'] == 281626]

,session_id,timestamp,item_id,category
824193,281626,2014-04-06T09:20:42.331Z,214535653.0,0.0
824194,281626,2014-04-06T09:22:05.402Z,214819357.0,0.0
824195,281626,2014-04-06T09:22:58.720Z,214535653.0,0.0
824196,281626,2014-04-06T09:23:15.534Z,214535653.0,0.0
824197,281626,2014-04-06T09:24:16.421Z,214821277.0,0.0
824198,281626,2014-04-06T09:26:07.764Z,214684513.0,0.0
824199,281626,2014-04-06T09:28:15.255Z,214535681.0,0.0
824200,281626,2014-04-06T09:29:00.938Z,214552370.0,0.0
824201,281626,2014-04-06T09:31:52.830Z,214698577.0,0.0


In [0]:
df.groupby('session_id')['item_id'].size()

session_id
1           4
2           6
3           3
4           2
6           2
           ..
4726778     2
4726779    10
4726781     2
4726782     3
4726783     3
Name: item_id, Length: 3728934, dtype: int64

In [0]:
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df['valid_session'].value_counts()

True     10659921
False     3352493
Name: valid_session, dtype: int64

In [0]:
df.loc[df.valid_session].shape

(10659921, 5)

In [0]:
# filter out item session with length > 2
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df = df.loc[df.valid_session].drop('valid_session',axis=1)
df.shape

(10659921, 4)

In [0]:
df [df['session_id'] == 4726778]

,session_id,timestamp,item_id,category


In [0]:
df.nunique()

session_id     1919301
timestamp     10644076
item_id          38546
category             1
dtype: int64

In [0]:
# #randomly sample a couple of them
sampled_session_id = np.random.choice(df.session_id.unique(), 100000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    100000
timestamp     555885
item_id        20464
category           1
dtype: int64

In [0]:
# average length of session 
df.groupby('session_id')['item_id'].size().mean()

5.55948

In [0]:
from sklearn.preprocessing import LabelEncoder

item_encoder = LabelEncoder()
category_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id )
df['category']= category_encoder.fit_transform(df.category.apply(str))
df.head()

,session_id,timestamp,item_id,category
208,61,2014-04-02T07:00:13.748Z,12932,0
209,61,2014-04-02T07:00:17.209Z,12932,0
210,61,2014-04-02T07:01:26.344Z,12932,0
211,61,2014-04-02T07:02:02.150Z,12932,0
212,61,2014-04-02T07:02:05.171Z,12932,0


In [0]:
df['item_id'].min(),df['item_id'].max()

(0, 20463)

In [0]:
#  check there is any buy event for a given session
buy_df = buy_df.loc[buy_df.session_id.isin(df.session_id)]
buy_df['item_id'] = item_encoder.transform(buy_df.item_id)
buy_df.head()

,session_id,timestamp,item_id,price,quantity
96,420457,2014-04-01T09:58:16.652Z,12566,7328,1
139,281789,2014-04-04T18:43:07.127Z,14177,2617,2
140,281789,2014-04-04T18:43:07.292Z,14178,2408,1
171,141007,2014-04-01T17:36:22.260Z,16089,941,1
172,141007,2014-04-01T17:36:22.277Z,4235,523,1


In [0]:
buy_df['session_id'].min(),buy_df['session_id'].max()

(737, 4726273)

In [0]:
buy_df['item_id'].min(),buy_df['item_id'].max()

(0, 20459)

In [0]:
buy_item_dict = dict(buy_df.groupby('session_id')['item_id'].apply(list))
buy_item_dict

{737: [17340],
 1046: [16321, 16322, 2987],
 1446: [15899, 15900, 15899],
 2398: [1009, 15975, 14830, 16032],
 2666: [12833, 12386],
 2998: [4828],
 3323: [5846, 19011, 19014, 19013],
 4873: [14168],
 4993: [17445],
 5496: [15910, 1095, 48, 15897],
 6447: [16045, 16049],
 6662: [14939, 15015],
 7717: [17060],
 7968: [11336, 6072, 6060],
 9568: [18173, 18477, 3560, 10476],
 10863: [16323, 16288, 8996, 13162],
 10906: [18973],
 11644: [14778, 2839, 15899, 7466],
 11833: [14162],
 12101: [15883, 16392],
 12714: [10400, 16084, 10400, 16084],
 13132: [16050, 16049],
 13297: [14443],
 13314: [16284, 16261, 14163, 14163],
 14819: [15957, 15755],
 14862: [16345, 16346, 16344],
 14883: [5490],
 14982: [325],
 16914: [15897, 15896, 15894],
 16949: [16038],
 17892: [10400],
 17938: [4944],
 18014: [15619, 15958],
 18191: [15884],
 18741: [3590],
 18774: [16045, 16309, 16047],
 19171: [4792],
 19349: [12622, 12622],
 19909: [3862],
 19983: [16365, 16365],
 21734: [15950, 16047, 16047, 15950],
 228

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
if 467077 in buy_item_dict:
    print("Hello")
    positive_indices = le.fit_transform(buy_item_dict[467077])
    print(positive_indices)


Hello
[5 0 3 2 1 4]


In [0]:
df.head()

,session_id,timestamp,item_id,category
208,61,2014-04-02T07:00:13.748Z,12932,0
209,61,2014-04-02T07:00:17.209Z,12932,0
210,61,2014-04-02T07:01:26.344Z,12932,0
211,61,2014-04-02T07:02:02.150Z,12932,0
212,61,2014-04-02T07:02:05.171Z,12932,0


In [0]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['/content/drive/My Drive/RecSys/yoochoose_click_binary_100000_sess.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            le = LabelEncoder()
            sess_item_id = le.fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id','category']].sort_values('sess_item_id')[['item_id','category']].drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            if session_id in buy_item_dict:
                positive_indices = le.transform(buy_item_dict[session_id])
                # print(positive_indices)
                label = np.zeros(len(node_features))
                label[positive_indices] = 1
            else:
                # print("Hello")
                label = [0] * len(node_features)


             = torch.FloatTensor(label)

            data = Data(x=x, edge_index=edge_index, y=y)
y
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        

In [0]:
dataset = YooChooseDataset('../')

In [0]:
dataset

YooChooseDataset(100000)

In [0]:
# grouped = df.groupby('session_id')
# data_list= []
# for session_id, group in tqdm(grouped):
#     le = LabelEncoder()
#     sess_item_id = le.fit_transform(group.item_id)
#     group = group.reset_index(drop=True)
#     group['sess_item_id'] = sess_item_id
#     node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

#     node_features = torch.LongTensor(node_features).unsqueeze(1)
#     target_nodes = group.sess_item_id.values[1:]
#     source_nodes = group.sess_item_id.values[:-1]

#     edge_index = torch.tensor([source_nodes,
#                            target_nodes], dtype=torch.long)
#     x = node_features
    
#     if session_id in buy_item_dict:
#         positive_indices = le.transform(buy_item_dict[session_id])
#         label = np.zeros(len(node_features))
#         label[positive_indices] = 1
#     else:
#         label = [0] * len(node_features)

    
#     y = torch.FloatTensor(label)
    
#     data = Data(x=x, edge_index=edge_index, y=y)
    
#     data_list.append(data)

In [0]:
dataset = dataset.shuffle()
one_tenth_length = int(len(dataset) * 0.1)
train_dataset = dataset[:one_tenth_length * 8]
val_dataset = dataset[one_tenth_length*8:one_tenth_length * 9]
test_dataset = dataset[one_tenth_length*9:]
len(train_dataset), len(val_dataset), len(test_dataset)

(80000, 10000, 10000)

In [0]:
from torch_geometric.data import DataLoader
batch_size= 512
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [0]:
num_items = df.item_id.max() +1
num_categories = df.category.max()+1
num_items , num_categories

(20464, 1)

In [0]:
embed_dim = 128
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GraphConv(embed_dim * 2, 128)
        self.pool1 = TopKPooling(128, ratio=0.9)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.9)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.9)
        self.item_embedding = torch.nn.Embedding(num_embeddings=num_items, embedding_dim=embed_dim)
        self.category_embedding = torch.nn.Embedding(num_embeddings=num_categories, embedding_dim=embed_dim)        
        self.lin1 = torch.nn.Linear(256, 256)
        self.lin2 = torch.nn.Linear(256, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        item_id = x[:,:,0]
        category = x[:,:,1]
        

        emb_item = self.item_embedding(item_id).squeeze(1)
        emb_category = self.category_embedding(category).squeeze(1)
        
#         emb_item = emb_item.squeeze(1)
#         emb_cat
        x = torch.cat([emb_item, emb_category], dim=1)  
#         print(x.shape)
        x = F.relu(self.conv1(x, edge_index))
#                 print(x.shape)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _= self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _= self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.act2(x)      
        
        outputs = []
        for i in range(x.size(0)):
            output = torch.matmul(emb_item[data.batch == i], x[i,:])

            outputs.append(output)
              
        x = torch.cat(outputs, dim=0)
        x = torch.sigmoid(x)
        
        return x

In [0]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
crit = torch.nn.BCELoss()

In [0]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [0]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    print(predictions)
    labels = np.hstack(labels)
    print(labels)
    
    return roc_auc_score(labels, predictions)

In [0]:
# print(predictions)

In [0]:
for epoch in range(1, 10):
    loss = train()
    # val_loader = DataLoader(val_dataset, batch_size=batch_size)
    # test_loader = DataLoader(test_dataset, batch_size=batch_size)   
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

[0.00122026 0.00585383 0.00050285 ... 0.05147558 0.00476769 0.01528503]
[0. 0. 0. ... 0. 0. 0.]
[5.4287102e-07 6.2577038e-07 2.7658456e-07 ... 7.2161420e-03 1.9137913e-05
 1.8409908e-05]
[0. 0. 0. ... 0. 0. 0.]
[0.05524515 0.28086868 0.04109479 ... 0.00393238 0.00151296 0.00211929]
[0. 0. 0. ... 0. 0. 0.]
Epoch: 001, Loss: 0.21372, Train Auc: 0.85757, Val Auc: 0.70654, Test Auc: 0.67754
[0.00040449 0.0053303  0.00022467 ... 0.01589412 0.00594789 0.00514106]
[0. 0. 0. ... 0. 0. 0.]
[6.8075046e-08 2.5775009e-07 6.6751603e-08 ... 3.0802111e-03 4.4780933e-05
 4.9193932e-05]
[0. 0. 0. ... 0. 0. 0.]
[0.07775791 0.14691028 0.11910281 ... 0.002631   0.00578548 0.00079163]
[0. 0. 0. ... 0. 0. 0.]
Epoch: 002, Loss: 0.20220, Train Auc: 0.87646, Val Auc: 0.71124, Test Auc: 0.68147
[5.9863261e-05 4.3468166e-04 3.3313008e-05 ... 1.9121578e-02 3.6570567e-03
 1.0476404e-02]
[0. 0. 0. ... 0. 0. 0.]
[6.3545915e-09 2.9846561e-08 8.7068281e-09 ... 2.2912987e-03 2.8538002e-06
 7.7541408e-06]
[0. 0. 0. ... 